# Chapter 14: Surrogate Models

In [6]:
import numpy as np
from itertools import product

## Algorithm 14.1

In [61]:
def design_matrix(X):
    n, m = len(X[0]), len(X)
    return np.array([ [1.0 if j == 0 else X[i][j-1] for j in range(n+1)]for i in range(m)])

def linear_regression(X,y):
    theta = np.linalg.solve(design_matrix(X),y)
    f = lambda x: np.matmul(theta,np.array([1]+x))
    return f

### Example

In [64]:
X = [[2,3,4],[3,2,4],[3,4,1],[3,4,3]]
y = [3,2,1,6]
model = linear_regression(X,y)
model([2,3,4])

[-21.25   2.25   3.25   2.5 ]


3.0

## Algorithm 14.2

In [ ]:
from itertools import product
def regression(X,y, bases):
    B =[ b(x) for x, b in product(X, bases) ]
    theta = np.linalg.solve(B, y)
    f = lambda x: np.sum([theta[i]*bases[i](x) for i in range(len(theta))])
    return f

## Algorithm 14.3

In [45]:
def polynomial_bases_1d(i,k):
    f = lambda x:[x[i]**p for p in range(k+1)]
    return f 
def polynomial_bases(n,k):
    bases = [ polynomial_bases_1d(i, k) for i in range(n)]
    terms = []
    for ks in product(*[np.arange(0,k+1) for i in range(n)]):

        if np.sum(ks) <= k:
            f = lambda x: np.prod(b(x) for b in bases )
            terms.append(f)
        break
    return terms